# ITT-MCT Schematic Model: Stress Relaxation

## Learning Objectives

1. Generate **synthetic relaxation data** from NB01 calibrated parameters
2. Understand **two-step relaxation** (β and α processes) in MCT
3. Analyze the **non-ergodicity parameter f** as the long-time plateau
4. Fit the model to verify parameter recovery
5. Compare fluid vs glass relaxation behavior

## Prerequisites

- **NB01: Schematic Flow Curve** (required for calibrated parameters)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~2-3 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~10-15 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.itt_mct import ITTMCTSchematic

# Add examples/utils to path
sys.path.insert(0, os.path.join("..", "utils"))
from itt_mct_tutorial_utils import (
    load_itt_mct_parameters,
    set_model_parameters,
    generate_synthetic_relaxation_schematic,
    save_itt_mct_results,
    print_convergence_summary,
    print_parameter_comparison,
    print_glass_state_summary,
    compute_fit_quality,
    compute_non_ergodicity_parameter,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Stress Relaxation in MCT

### Two-Step Relaxation

Mode-Coupling Theory predicts a characteristic **two-step relaxation**:

1. **Fast β-process**: Initial decay from Φ(0)=1 toward plateau
   - Time scale: τ_β ~ 1/Γ (bare relaxation)
   - Power-law: Φ(t) ~ f + A t^(-a)

2. **Slow α-process**: Final decay from plateau to equilibrium
   - Time scale: τ_α diverges at glass transition
   - Stretched exponential: Φ(t) ~ exp(-(t/τ_α)^β)

### Key Equations

**Correlator equation** (schematic):
$$
\frac{\partial \Phi}{\partial t} + \Gamma\left[\Phi + \int_0^t m(\Phi) \frac{\partial \Phi}{\partial s} ds\right] = 0
$$

**Stress relaxation** after step strain γ₀:
$$
\sigma(t) = G_\infty \gamma_0 \Phi(t)^2 h(\gamma_0)
$$

### Fluid vs Glass

| Property | Fluid (ε < 0) | Glass (ε > 0) |
|----------|---------------|---------------|
| Long-time limit | Φ(∞) = 0 | Φ(∞) = f > 0 |
| α-relaxation | Finite τ_α | τ_α → ∞ |
| Final stress | σ(∞) = 0 | σ(∞) = G∞ γ₀ f² h(γ₀) |

### MCT Exponents

Near the glass transition:
- β-relaxation: Φ ~ f + A t^(-a) with a ≈ 0.3
- α-relaxation: Φ ~ f × exp(-(t/τ_α)^b) with b ≈ 0.5

## 3. Load Calibrated Parameters from NB01

In [ ]:
# Load parameters calibrated in NB01
try:
    params = load_itt_mct_parameters("schematic", "flow_curve")
    print("Loaded parameters from NB01:")
    for name, val in params.items():
        print(f"  {name:10s} = {val:.4g}")
except FileNotFoundError as e:
    print(f"Warning: {e}")
    print("Using default parameters (run NB01 first for calibrated values)")
    params = {"v2": 4.2, "Gamma": 1.0, "gamma_c": 0.1, "G_inf": 1000.0}

In [ ]:
# Create model and set parameters
model = ITTMCTSchematic(epsilon=0.05)  # Initialize in glass state
set_model_parameters(model, params)

print("\nModel state:")
print(model)
print()
print_glass_state_summary(model)

## 4. Generate Synthetic Relaxation Data

In [ ]:
# Generate synthetic data with noise
SIGMA_0 = 100.0  # Initial stress (Pa)
T_END = 100.0    # End time (s)
NOISE_LEVEL = 0.02  # 2% noise

time_data, stress_data = generate_synthetic_relaxation_schematic(
    model,
    sigma_0=SIGMA_0,
    t_end=T_END,
    n_points=200,
    noise_level=NOISE_LEVEL,
    seed=42,
)

print(f"Generated {len(time_data)} data points")
print(f"Time range: [{time_data.min():.4f}, {time_data.max():.2f}] s")
print(f"Stress range: [{stress_data.min():.2f}, {stress_data.max():.2f}] Pa")

In [ ]:
# Plot synthetic data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Linear scale
ax1.plot(time_data, stress_data, "ko", markersize=4, alpha=0.7)
ax1.set_xlabel("Time [s]", fontsize=12)
ax1.set_ylabel("Stress [Pa]", fontsize=12)
ax1.set_title("Stress Relaxation (Linear)", fontsize=13)
ax1.grid(True, alpha=0.3)

# Right: Log-log scale
ax2.loglog(time_data, stress_data, "ko", markersize=4, alpha=0.7)
ax2.set_xlabel("Time [s]", fontsize=12)
ax2.set_ylabel("Stress [Pa]", fontsize=12)
ax2.set_title("Stress Relaxation (Log-Log)", fontsize=13)
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. NLSQ Fitting

In [ ]:
# Compute pre-shear strain for fitting
G_inf = model.parameters.get_value("G_inf")
gamma_pre = SIGMA_0 / G_inf
print(f"Pre-shear strain: γ₀ = σ₀/G∞ = {gamma_pre:.4f}")

In [ ]:
# Fit to relaxation data
param_names = ["v2", "Gamma", "gamma_c", "G_inf"]

t0 = time.time()
model.fit(time_data, stress_data, test_mode="relaxation", gamma_pre=gamma_pre, method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    orig = params.get(name, val)
    print(f"  {name:10s} = {val:.4g}  (original: {orig:.4g})")

In [ ]:
# Compute fit quality
stress_pred = model.predict(time_data, test_mode="relaxation", gamma_pre=gamma_pre)
metrics = compute_fit_quality(stress_data, stress_pred)

print(f"\nFit Quality:")
print(f"  R²:   {metrics['R2']:.6f}")
print(f"  RMSE: {metrics['RMSE']:.4g} Pa")
print(f"  NRMSE: {metrics['NRMSE']:.4%}")

In [ ]:
# Plot fit
time_fine = np.logspace(-2, np.log10(T_END), 200)
stress_pred_fine = model.predict(time_fine, test_mode="relaxation", gamma_pre=gamma_pre)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Linear
ax1.plot(time_data, stress_data, "ko", markersize=5, label="Synthetic data")
ax1.plot(time_fine, stress_pred_fine, "-", lw=2, color="C0", label="F₁₂ fit")
ax1.set_xlabel("Time [s]", fontsize=12)
ax1.set_ylabel("Stress [Pa]", fontsize=12)
ax1.set_title(f"Stress Relaxation Fit (R² = {metrics['R2']:.4f})", fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Right: Log-log
ax2.loglog(time_data, stress_data, "ko", markersize=5, label="Synthetic data")
ax2.loglog(time_fine, stress_pred_fine, "-", lw=2, color="C0", label="F₁₂ fit")

# Mark residual stress (glass plateau)
info = model.get_glass_transition_info()
if info["is_glass"]:
    f_neq = info.get("f_neq", 0.3)
    gamma_c = model.parameters.get_value("gamma_c")
    h_gamma = np.exp(-(gamma_pre/gamma_c)**2)
    sigma_residual = G_inf * gamma_pre * f_neq**2 * h_gamma
    ax2.axhline(sigma_residual, color="red", linestyle="--", alpha=0.5,
                label=f"σ_res ≈ {sigma_residual:.1f} Pa")

ax2.set_xlabel("Time [s]", fontsize=12)
ax2.set_ylabel("Stress [Pa]", fontsize=12)
ax2.set_title("Log-Log Scale", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

### 5.1 Physical Analysis: Two-Step Relaxation

In [ ]:
# Analyze two-step relaxation
print("Two-Step Relaxation Analysis")
print("=" * 50)

v2 = model.parameters.get_value("v2")
Gamma = model.parameters.get_value("Gamma")
f_neq = compute_non_ergodicity_parameter(0, v2)

# Time scales
tau_beta = 1.0 / Gamma  # β-relaxation

print(f"\nβ-process (fast):")
print(f"  Time scale: τ_β = 1/Γ = {tau_beta:.4g} s")
print(f"  Physics: Rattling within cage")

print(f"\nα-process (slow):")
if info["is_glass"]:
    print(f"  Time scale: τ_α → ∞ (glass state)")
    print(f"  Physics: Cage breaking arrested")
else:
    # Estimate alpha relaxation time
    epsilon = (v2 - 4) / 4
    tau_alpha = tau_beta * abs(epsilon)**(-2.5) if epsilon < 0 else float('inf')
    print(f"  Time scale: τ_α ≈ {tau_alpha:.4g} s")
    print(f"  Physics: Cage rearrangement")

print(f"\nPlateau:")
print(f"  Non-ergodicity parameter: f = {f_neq:.4f}")
if info["is_glass"]:
    print(f"  Stress plateau: σ_∞ > 0 (residual stress)")
else:
    print(f"  Stress plateau: σ_∞ = 0 (complete relaxation)")

In [ ]:
# Compare fluid vs glass relaxation
fig, ax = plt.subplots(figsize=(10, 6))

# Glass state (current)
ax.loglog(time_fine, stress_pred_fine, "-", lw=2.5, color="C0", 
          label=f"Glass (ε = {info['epsilon']:.3f})")

# Simulate fluid state
model_fluid = ITTMCTSchematic(epsilon=-0.1)  # Fluid
set_model_parameters(model_fluid, params)
model_fluid.parameters.set_value("v2", 3.6)  # Below critical
stress_fluid = model_fluid.predict(time_fine, test_mode="relaxation", gamma_pre=gamma_pre)
ax.loglog(time_fine, stress_fluid, "--", lw=2, color="C1", 
          label="Fluid (ε = -0.1)")

ax.axhline(1e-1, color="gray", linestyle=":", alpha=0.5)
ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Relaxation: Glass vs Fluid", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Bayesian Inference

In [ ]:
# Prepare warm-start
initial_values = {
    name: model.parameters.get_value(name)
    for name in param_names
}

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    time_data,
    stress_data,
    test_mode="relaxation",
    gamma_pre=gamma_pre,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
all_pass = print_convergence_summary(result, param_names)

In [ ]:
# Trace plots
idata = result.to_inference_data()
axes = az.plot_trace(idata, var_names=param_names, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Relaxation)", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Parameter comparison
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, param_names)

## 7. Physical Interpretation

### MCT Relaxation Signatures

1. **β-relaxation** (t ~ τ_β):
   - Fast decay due to particle vibrations within cage
   - Followed by plateau toward non-ergodicity parameter f

2. **α-relaxation** (t ~ τ_α):
   - Slow decay from cage rearrangement
   - In glass: τ_α → ∞, stress plateaus
   - In fluid: finite τ_α, stress decays to zero

3. **Residual stress** (glass only):
   - σ_∞ = G∞ × γ₀ × f² × h(γ₀)
   - Non-zero due to arrested dynamics

In [ ]:
# Summary
print("Stress Relaxation Summary")
print("=" * 50)
print_glass_state_summary(model)
print(f"\nRelaxation Characteristics:")
print(f"  Initial stress: σ₀ = {SIGMA_0:.1f} Pa")
print(f"  Pre-shear strain: γ₀ = {gamma_pre:.4f}")
print(f"  β time scale: τ_β = {1/Gamma:.4g} s")
if info["is_glass"]:
    print(f"  α time scale: τ_α → ∞ (glass)")
    print(f"  Residual stress: σ_∞ > 0")

## 8. Save Results

In [ ]:
# Save results
save_itt_mct_results(model, result, "schematic", "relaxation", param_names)
print("\nRelaxation results saved.")

## Key Takeaways

1. **Two-step relaxation** is a hallmark of MCT:
   - Fast β-process (cage vibration)
   - Slow α-process (cage rearrangement)

2. **Glass state** (ε > 0) shows:
   - Residual stress σ_∞ > 0
   - Plateau at non-ergodicity parameter f
   - Arrested α-relaxation (τ_α → ∞)

3. **Fluid state** (ε < 0) shows:
   - Complete relaxation to σ_∞ = 0
   - Finite α-relaxation time τ_α

4. **Parameter recovery** from synthetic data validates model

### Next Steps

- **NB04:** Creep (delayed yielding)
- **NB05:** SAOS (G' plateau from f)